In [2]:
import numpy as np
import pandas as pd
import os
from openpyxl import utils
import warnings
import statistics
warnings.filterwarnings("ignore", category=UserWarning)
from scipy.stats import fisher_exact

In [3]:
统计 = pd.read_excel('C:/Users/NC-O10/Dropbox/云蜚科技/脂质紫杉醇/紫杉醇干净的数据.xlsx', sheet_name='统计用100例')
交差 = pd.read_excel('C:/Users/NC-O10/Dropbox/云蜚科技/脂质紫杉醇/紫杉醇干净的数据.xlsx', sheet_name='交差100例')

In [ ]:
## 一. 描述性分析

In [ ]:
#年龄recoding
from datetime import datetime
出生日期= 基本信息['出生日期']
# Sample data with a column of birth dates
birth_dates = 基本信息['出生日期']

# Function to calculate age from a birth date
def calculate_age(birth_date):
    current_date = datetime.now().date()
    birth_date = datetime.strptime(birth_date, '%Y-%m-%d').date()
    age = current_date.year - birth_date.year

    if current_date.month < birth_date.month or (current_date.month == birth_date.month and current_date.day < birth_date.day):
        age -= 1

    return age

# Convert birth dates to ages
ages = [calculate_age(birth_date) for birth_date in birth_dates]

# Print the resulting ages
#print("Ages:", ages)
基本信息['年龄']=ages
中位数_年龄=statistics.mean(基本信息['年龄'])
范围_年龄max=max(基本信息['年龄'])
范围_年龄min=min(基本信息['年龄'])
print(中位数_年龄)
print(范围_年龄min)
print(范围_年龄max)
#recode 年龄 greater than mean=1, lower than mean=0
# Assuming you have a DataFrame with an 'Age' column
年龄10 = pd.DataFrame({'Age': 基本信息['年龄']})

# Calculate the mean age
mean_age = np.mean(年龄10['Age'])

# Create a new column for recoded categories
年龄10['Age_Category'] = ''

# Recode ages based on mean
年龄10.loc[年龄10['Age'] > mean_age, 'Age_Category'] = '1'
年龄10.loc[年龄10['Age'] < mean_age, 'Age_Category'] = '0'

# Print the updated DataFrame
#print(年龄10)

#calculate the fisher's exact test p-value


from scipy.stats import fisher_exact
import pandas as pd

# Assuming you have a DataFrame with exposure and outcome columns
紫杉醇_年龄 = pd.DataFrame({'Exposure': 年龄10['Age_Category'],
                   'Outcome': 紫杉醇['紫杉醇脂质体是否一线治疗']})

# Extract the exposure and outcome columns as separate variables
exposure = 紫杉醇_年龄['Exposure']
outcome = 紫杉醇_年龄['Outcome']

# Perform Fisher's exact test
odds_ratio, p_value = fisher_exact(pd.crosstab(exposure, outcome))

# Print the results
#print("Odds Ratio:", odds_ratio)
print("p-value:", p_value)